In [2]:
import numpy as np
import pandas as pd
from mediapipe.tasks import python
from mediapipe.tasks.python import text
from parsivar.tokenizer import Tokenizer

In [4]:
df = pd.read_excel('comment.xlsx', usecols=['comment'])
df.head()

,comment
0,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...
1,بوی تند ولی خوشبو داره.ماندگاریش خوبه و هر چقد...
2,متاسفانه عمر مفید این ماشین کم هست و بعد از دو...
3,افا از وقتی من اینو واسه پسرم خریدم همه بچه ها...
4,این تیغ محصول کشور آمریکا هست ( در عکس محصول م...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100008 entries, 0 to 100007
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  99903 non-null  object
dtypes: object(1)
memory usage: 781.4+ KB


In [6]:
sentences = []

def tokenizer(text):
    tokenizer = Tokenizer()
    tokens = tokenizer.tokenize_sentences(str(text))
    
    for token in tokens:
        sentences.append(token)

df['comment'].apply(tokenizer)

0         None
1         None
2         None
3         None
4         None
          ... 
100003    None
100004    None
100005    None
100006    None
100007    None
Name: comment, Length: 100008, dtype: object

In [7]:
sentences_df = pd.DataFrame(sentences, columns=['comment'])
sentences_df.head()

,comment
0,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...
1,پس بهتره ک نظر غیر کارشناسی و الکی ندیم .
2,من ۲سال این پاور بانک رو دارم، برای ۲نفر اشنا ...
3,همزمان با این پاور یک adata 10000 هم برای همسر...
4,طبق تجربه من و اطرافیانم ک شیائومی داشتن، اگر ...


In [8]:
base_options = python.BaseOptions(model_asset_path='bert_embedder.tflite')

l2_normalize = True
quantize = True

options = text.TextEmbedderOptions(base_options=base_options, l2_normalize=l2_normalize, quantize=quantize)

In [14]:
def embedding(comment):
    with text.TextEmbedder.create_from_options(options) as embedder:
        embedding_result = embedder.embed(comment)
        return embedding_result.embeddings[0].embedding
    
sample_embedded_comments = sentences_df['comment'].iloc[0:1000].apply(embedding)

In [15]:
embedded_df = pd.DataFrame(sample_embedded_comments.tolist())
embedded_df.head()

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,10,250,1,5,253,251,255,7,12,6,...,0,9,4,0,255,13,3,255,255,247
1,12,249,253,251,0,251,5,8,12,11,...,3,14,4,253,255,9,255,252,0,241
2,11,249,0,5,253,249,0,8,13,4,...,0,6,5,0,254,12,2,0,255,246
3,13,250,254,255,255,248,3,8,13,8,...,1,11,3,254,254,10,1,252,0,242
4,13,252,253,2,254,248,2,8,11,6,...,0,10,3,253,254,10,4,254,255,244


In [23]:
from sklearn.cluster import HDBSCAN

#Not Completed
dbscan = HDBSCAN(min_samples=50, n_jobs=-1)
labels = dbscan.fit_predict(embedded_df)

final_df = pd.DataFrame({'label': labels, 'comment': sentences_df['comment'].iloc[0:1000]})
final_df

,label,comment
0,0,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...
1,0,پس بهتره ک نظر غیر کارشناسی و الکی ندیم .
2,0,من ۲سال این پاور بانک رو دارم، برای ۲نفر اشنا ...
3,0,همزمان با این پاور یک adata 10000 هم برای همسر...
4,0,طبق تجربه من و اطرافیانم ک شیائومی داشتن، اگر ...
...,...,...
995,-1,باسلام .
996,0,نسبت به قیمتی که داره بسیار عالی و مقرون به ص...
997,0,گوشی خوبیه یکساله دارم تا حالا به مشکل نخوردم ...
998,1,اگه برای کارای روزمره مث زنگ زدن و پیام دادن و...
